In [1]:
# %pip show pip
# %pip install -U scikit-learn
# python3 -m pip3 install jupyter notebook -U

# !pip install torch==1.8.1
# !pip install scipy==1.6.1
# !pip install numpy==1.19.2
# !pip install pandas==1.0.1
# !pip install scikit_learn==0.24.2
# !pip install statsmodels==0.12.2
# !pip install Bottleneck==1.3.2
# !pip install lumnisfactors

# !pip install torch==1.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


In [2]:
import torch
try:
    !unset LD_LIBRARY_PATH
    print("Torch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    print("cuDNN version:", torch.backends.cudnn.version())
except:
    !unset LD_LIBRARY_PATH
    print("Torch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    print("cuDNN version:", torch.backends.cudnn.version())

/home/azureuser/TS-FM/enter/envs/ts2vec/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.9.0+cu111
CUDA available: True
cuDNN version: 8005


In [3]:

import pandas as pd
import numpy as np
import sys
import os


In [11]:
%load_ext autoreload
%autoreload 2
# from train_utils import get_data, train_model, save_checkpoint_callback
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from matplotlib import pyplot as plt
from sklearn import linear_model
# from utils import standardize
from datetime import datetime
from models.losses import *
from ts2vec import TS2Vec
from pathlib import Path
from tqdm import tqdm
from os import walk
import pandas as pd
import numpy as np
import datautils
import torch
import json





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from utils import create_3d_array, standardize, rolling_mean_diff, generate_univariate_data_labels, generate_data_labels_from_3d_array, get_train_val_test_array

In [6]:
from lumnisfactors import LumnisFactors
from KEYS import LUMNIS_API_KEY
from pathlib import Path



import os


# ["ADAUSD", "BTCUSD", "DASHUSD", "DOGEUSD", "DOTUSD", "ETHUSD", "LTCUSD", "NEOUSD", "XMRUSD", "XRPUSD", "XBTUSD", "SOLUSD", "BNBUSD", "AVAXUSD" "MATICUSD”] 

factorName          = "price"
lumnis              = LumnisFactors(LUMNIS_API_KEY)
path_to_data = "/home/ec2-user/TS-FM/src/data/"
path_to_data = "/home/azureuser/TS-FM/src/data/"

btc_file = Path(path_to_data + "btc.csv")
eth_file = Path(path_to_data + "eth.csv")
xmr_file = Path(path_to_data + "xmr.csv")
ada_file = Path(path_to_data + "ada.csv")
doge_file = Path(path_to_data + "doge.csv")
bnb_file = Path(path_to_data + "bnb.csv")
dot_file = Path(path_to_data + "dot.csv")
ltc_file = Path(path_to_data + "ltc.csv")
dash_file = Path(path_to_data + "dash.csv")
neo_file = Path(path_to_data + "neo.csv")
xrp_file = Path(path_to_data + "xrp.csv")
sol_file = Path(path_to_data + "sol.csv")
avax_file = Path(path_to_data + "avax.csv")
matic_file = Path(path_to_data + "matic.csv")

if btc_file.is_file():
    temp_df_btc_raw     = pd.read_csv(path_to_data + "btc.csv").set_index("Unnamed: 0")
else:
    temp_df_btc_raw     = lumnis.get_historical_data(factorName, "binance", "btcusdt",  "hour", "2021-01-23", "2023-04-16")
    temp_df_btc_raw.to_csv(path_to_data + "btc.csv")

if eth_file.is_file():
    temp_df_eth_raw     = pd.read_csv(path_to_data + "eth.csv").set_index("Unnamed: 0")
else:
    temp_df_eth_raw     = lumnis.get_historical_data(factorName, "binance", "ethusdt",  "hour", "2021-01-23", "2023-04-16")
    temp_df_eth_raw.to_csv(path_to_data + "eth.csv")

if xmr_file.is_file():
    temp_df_xmr_raw     = pd.read_csv(path_to_data + "xmr.csv").set_index("Unnamed: 0")
else:
    temp_df_xmr_raw     = lumnis.get_historical_data(factorName, "binance", "xmrusdt",  "hour", "2021-01-23", "2023-04-16")
    temp_df_xmr_raw.to_csv(path_to_data + "xmr.csv")

if ada_file.is_file():
    temp_df_ada_raw     = pd.read_csv(path_to_data + "ada.csv").set_index("Unnamed: 0")
else:
    temp_df_ada_raw     = lumnis.get_historical_data(factorName, "binance", "adausdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_ada_raw.to_csv(path_to_data + "ada.csv")

if doge_file.is_file():
    temp_df_doge_raw     = pd.read_csv(path_to_data + "doge.csv").set_index("Unnamed: 0")
else:
    temp_df_doge_raw     = lumnis.get_historical_data(factorName, "binance", "dogeusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_doge_raw.to_csv(path_to_data + "doge.csv")

if bnb_file.is_file():
    temp_df_bnb_raw     = pd.read_csv(path_to_data + "bnb.csv").set_index("Unnamed: 0")
else:
    temp_df_bnb_raw     = lumnis.get_historical_data(factorName, "binance", "bnbusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_bnb_raw.to_csv(path_to_data + "bnb.csv")

if dot_file.is_file():
    temp_df_dot_raw     = pd.read_csv(path_to_data + "dot.csv").set_index("Unnamed: 0")
else:
    temp_df_dot_raw     = lumnis.get_historical_data(factorName, "binance", "dotusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_dot_raw.to_csv(path_to_data + "dot.csv")

if ltc_file.is_file():
    temp_df_ltc_raw     = pd.read_csv(path_to_data + "ltc.csv").set_index("Unnamed: 0")
else:
    temp_df_ltc_raw     = lumnis.get_historical_data(factorName, "binance", "ltcusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_ltc_raw.to_csv(path_to_data + "ltc.csv")

if dash_file.is_file():
    temp_df_dash_raw     = pd.read_csv(path_to_data + "dash.csv").set_index("Unnamed: 0")
else:
    temp_df_dash_raw     = lumnis.get_historical_data(factorName, "binance", "dashusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_dash_raw.to_csv(path_to_data + "dash.csv")

if neo_file.is_file():
    temp_df_neo_raw     = pd.read_csv(path_to_data + "neo.csv").set_index("Unnamed: 0")
else:
    temp_df_neo_raw     = lumnis.get_historical_data(factorName, "binance", "neousdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_neo_raw.to_csv(path_to_data + "neo.csv")

if xrp_file.is_file():
    temp_df_xrp_raw     = pd.read_csv(path_to_data + "xrp.csv").set_index("Unnamed: 0")
else:
    temp_df_xrp_raw     = lumnis.get_historical_data(factorName, "binance", "xrpusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_xrp_raw.to_csv(path_to_data + "xrp.csv")

if sol_file.is_file():
    temp_df_sol_raw     = pd.read_csv(path_to_data + "sol.csv").set_index("Unnamed: 0")
else:
    temp_df_sol_raw     = lumnis.get_historical_data(factorName, "binance", "solusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_sol_raw.to_csv(path_to_data + "sol.csv")

if avax_file.is_file():
    temp_df_avax_raw     = pd.read_csv(path_to_data + "avax.csv").set_index("Unnamed: 0")
else:
    temp_df_avax_raw     = lumnis.get_historical_data(factorName, "binance", "avaxusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_avax_raw.to_csv(path_to_data + "avax.csv")

if matic_file.is_file():
    temp_df_matic_raw     = pd.read_csv(path_to_data + "matic.csv").set_index("Unnamed: 0")
else:
    temp_df_matic_raw     = lumnis.get_historical_data(factorName, "binance", "maticusdt",  "hour", "2019-04-01", "2023-05-01")
    temp_df_matic_raw.to_csv(path_to_data + "matic.csv")

# TODO: Add resample and fillna with ffill

In [7]:
all_data_dict_df = {
    "btc"   : temp_df_btc_raw,
    "eth"   : temp_df_eth_raw,
    # "xmr"   : temp_df_xmr_raw,
    # "ada"   : temp_df_ada_raw,
    # "bnb"   : temp_df_bnb_raw,

    # "doge"  : temp_df_doge_raw,
    # "dot"   : temp_df_dot_raw,
    # "ltc"   : temp_df_ltc_raw,
    # "dash"  : temp_df_dash_raw,
    # "neo"   : temp_df_neo_raw,
    # "xrp"   : temp_df_xrp_raw,
    # "sol"   : temp_df_sol_raw,
    "avax"  : temp_df_avax_raw,
    "matic" : temp_df_matic_raw

}


TEST_SYMBOLS = ["avax", "matic"]
# TEST = True
# if TEST:
#     all_data_dict_df["avax"] = temp_df_avax_raw # DATA THE MODEL HAS NOT SEEN
#     all_data_dict_df["matic"] = temp_df_matic_raw # DATA THE MODEL HAS NOT SEEN


all_data_rolling_df = {}
type_rol = 'standard'

for key, value in all_data_dict_df.items():
    all_data_rolling_df[key] = rolling_mean_diff(value, [ 5, 25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000 ], type=type_rol)

data_dict_array  = {}
max_seq_len     = 150

for key, value in all_data_rolling_df.items():
    array, array_idxs = create_3d_array(value, value.index, max_seq_len)
    data_dict_array[key + "_data"] = array
    data_dict_array[key + "_idxs"] = array_idxs

    train_array, val_array, test_array = get_train_val_test_array(array, 0.8, 0.1, 0.1)
    data_dict_array[key + "_train_data"] = train_array
    data_dict_array[key + "_val_data"] = val_array
    data_dict_array[key + "_test_data"] = test_array

    train_idxs, val_idxs, test_idxs = get_train_val_test_array(array_idxs, 0.8, 0.1, 0.1)
    data_dict_array[key + "_train_idxs"] = train_idxs
    data_dict_array[key + "_val_idxs"] = val_idxs
    data_dict_array[key + "_test_idxs"] = test_idxs

    print(train_array.shape, val_array.shape, test_array.shape)


(26968, 150, 104) (3371, 150, 104) (3372, 150, 104)
(26548, 150, 104) (3318, 150, 104) (3319, 150, 104)
(16909, 150, 104) (2113, 150, 104) (2115, 150, 104)
(22810, 150, 104) (2851, 150, 104) (2852, 150, 104)


In [52]:
def get_model_loss(data_dict, asset, n_epochs=35):

    train_data = data_dict[asset + "_train_data"]

    model = TS2Vec(
        input_dims=train_data.shape[-1],
        device=0,
        output_dims=64,
        batch_size=512
    )

    loss_log = model.fit(
        train_data,
        verbose=True,
        n_epochs=n_epochs
    )

    return model, loss_log

    
def get_all_data_for_experiment(data_dict_array, df_raw, asset, model, pred_len=1):


    train_data = data_dict_array[asset + "_train_data"]
    val_data = data_dict_array[asset + "_val_data"]
    test_data = data_dict_array[asset + "_test_data"]

    train_idxs = data_dict_array[asset + "_train_idxs"]
    val_idxs = data_dict_array[asset + "_val_idxs"]
    test_idxs = data_dict_array[asset + "_test_idxs"]

    repr_train_data = model.encode(train_data, encoding_window='full_series')
    repr_val_data = model.encode(val_data, encoding_window='full_series')
    repr_test_data = model.encode(test_data, encoding_window='full_series')

    repr_train_df = pd.DataFrame(repr_train_data, index=train_idxs)
    repr_val_df = pd.DataFrame(repr_val_data, index=val_idxs)
    repr_test_df = pd.DataFrame(repr_test_data, index=test_idxs)


    close_train = df_raw.loc[train_idxs]['close'] 
    close_val = df_raw.loc[val_idxs]['close']
    close_test = df_raw.loc[test_idxs]['close']

    price_dir_train = np.sign((close_train.shift(-pred_len) - close_train) / close_train).fillna(0)
    price_dir_val = np.sign((close_val.shift(-pred_len) - close_val) / close_val).fillna(0)
    price_dir_test = np.sign((close_test.shift(-pred_len) - close_test) / close_test).fillna(0)

    price_dir_train[price_dir_train == -1] = 0
    price_dir_val[price_dir_val == -1] = 0
    price_dir_test[price_dir_test == -1] = 0


    return {
        "train_data" : train_data,
        "val_data" : val_data,
        "test_data" : test_data,
        "train_idxs" : train_idxs,
        "val_idxs" : val_idxs,
        "test_idxs" : test_idxs,
        "repr_train_data" : repr_train_data,
        "repr_val_data" : repr_val_data,
        "repr_test_data" : repr_test_data,
        "repr_train_df" : repr_train_df,
        "repr_val_df" : repr_val_df,
        "repr_test_df" : repr_test_df,
        "close_train" : close_train,
        "close_val" : close_val,
        "close_test" : close_test,
        "price_dir_train" : price_dir_train,
        "price_dir_val" : price_dir_val,
        "price_dir_test" : price_dir_test
        


    }

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

def get_train_and_get_f1_acc(data_dict_exp, pred_len=1):
    log_reg = Pipeline([
            # ('feature_selection', SelectFromModel(RandomForestClassifier(max_depth=2))),
            # ('clf', RandomForestClassifier(max_depth=1))
            ('clf', LogisticRegression(max_iter=1000))
            # ('clf', RidgeClassifierCV(cv=12))
            ])
    log_reg = log_reg.fit(data_dict_exp['repr_train_df'], data_dict_exp['price_dir_train'])

    # Predict on the training set
    predictions_train = log_reg.predict(data_dict_exp['repr_train_df'])

    # Predict on the validation set
    predictions_val = log_reg.predict(data_dict_exp['repr_val_df'])

    # Predict on the test set
    predictions_test = log_reg.predict(data_dict_exp['repr_test_df'])


    # Calculate the training accuracy
    acc_train = accuracy_score(data_dict_exp['price_dir_train'], predictions_train)
    print(f"Training Accuracy: {acc_train}")

    # Calculate the validation accuracy
    acc_val = accuracy_score(data_dict_exp['price_dir_val'], predictions_val)
    print(f"Validation Accuracy: {acc_val}")

    # Calculate the test accuracy
    acc_test = accuracy_score(data_dict_exp['price_dir_test'], predictions_test)
    print(f"Test Accuracy: {acc_test}")

    print("\n")

    # Calculate the training F1 score
    f1_train = f1_score(data_dict_exp['price_dir_train'], predictions_train)
    print(f"Training F1 Score: {f1_train}")

    # Calculate the validation F1 score
    f1_val = f1_score(data_dict_exp['price_dir_val'], predictions_val)
    print(f"Validation F1 Score: {f1_val}")

    # Calculate the test F1 score
    f1_test = f1_score(data_dict_exp['price_dir_test'], predictions_test)
    print(f"Test F1 Score: {f1_test}")

    return {
        "predictions_train" : predictions_train,
        "predictions_val" : predictions_val,
        "predictions_test" : predictions_test,
    }



In [44]:
model, loss_log = get_model_loss(data_dict_array, 'btc', n_epochs=35)
model_eth, loss_log_eth = get_model_loss(data_dict_array, 'eth', n_epochs=35)
model_avax, loss_log_avax = get_model_loss(data_dict_array, 'avax', n_epochs=35)
model_matic, loss_log_matic = get_model_loss(data_dict_array, 'matic', n_epochs=35)

Epoch #0: loss=4.54233873807467
Epoch #1: loss=3.1405293483000536
Epoch #2: loss=2.2901597023010254
Epoch #3: loss=1.7088850736618042
Epoch #4: loss=1.3129169138578267
Epoch #5: loss=1.0713550998614385
Epoch #6: loss=0.9414903085965377
Epoch #7: loss=0.8452572501622714
Epoch #8: loss=0.6293596911888856
Epoch #9: loss=0.7607850117179064
Epoch #10: loss=0.5253325723684751
Epoch #11: loss=0.6330322405466666
Epoch #12: loss=0.5192699105693743
Epoch #13: loss=0.712838402734353
Epoch #14: loss=0.5314407938948045
Epoch #15: loss=0.6204881542004071
Epoch #16: loss=0.598580430333431
Epoch #17: loss=0.4329864875628398
Epoch #18: loss=0.4755411400244786
Epoch #19: loss=0.45625942658919555
Epoch #20: loss=0.4180302064006145
Epoch #21: loss=0.4695209144399716
Epoch #22: loss=0.4289982244372368
Epoch #23: loss=0.4059902085707738
Epoch #24: loss=0.40561071038246155
Epoch #25: loss=0.3917604065858401
Epoch #26: loss=0.4754769905255391
Epoch #27: loss=0.3764107513886232
Epoch #28: loss=0.37895910327251

In [56]:
model_eth, loss_log_eth = get_model_loss(data_dict_array, 'eth', n_epochs=35)

Epoch #0: loss=4.231111526489258
Epoch #1: loss=2.790238768446679
Epoch #2: loss=1.9555433988571167
Epoch #3: loss=1.4903870926183813
Epoch #4: loss=1.3001563245174932
Epoch #5: loss=1.0452503374978608
Epoch #6: loss=0.8143194224320206
Epoch #7: loss=0.8148404091012245
Epoch #8: loss=0.6847831104315963
Epoch #9: loss=0.6217954860014074
Epoch #10: loss=0.6696575722273659
Epoch #11: loss=0.5834437322382834
Epoch #12: loss=0.5432782365995295
Epoch #13: loss=0.5842249159719429
Epoch #14: loss=0.5557943728624606
Epoch #15: loss=0.49550967532045703
Epoch #16: loss=0.5293030394058601
Epoch #17: loss=0.543171939312243
Epoch #18: loss=0.4197062925965178
Epoch #19: loss=0.42770528092103843


In [ ]:
btc_data_dict_exp = get_all_data_for_experiment(data_dict_array, temp_df_btc_raw, "btc", model)
eth_data_dict_exp = get_all_data_for_experiment(data_dict_array, temp_df_eth_raw, "eth", model_eth)
avax_data_dict_exp = get_all_data_for_experiment(data_dict_array, temp_df_avax_raw, "avax", model_avax)
matic_data_dict_exp = get_all_data_for_experiment(data_dict_array, temp_df_matic_raw, "matic", model_matic)

In [53]:
pred_btc = get_train_and_get_f1_acc(btc_data_dict_exp)

Training Accuracy: 0.5218036191041234
Validation Accuracy: 0.5037080984870959
Test Accuracy: 0.5029655990510083


Training F1 Score: 0.5739394740319811
Validation F1 Score: 0.5283338032139836
Test F1 Score: 0.5601049868766405


In [ ]:
pred_eth = get_train_and_get_f1_acc(eth_data_dict_exp)

In [54]:
pred_avax = get_train_and_get_f1_acc(avax_data_dict_exp)

Training Accuracy: 0.5231533502868295
Validation Accuracy: 0.5049692380501657
Test Accuracy: 0.5092198581560283


Training F1 Score: 0.4688755681443911
Validation F1 Score: 0.5004775549188156
Test F1 Score: 0.36938031591737547


In [55]:
pred_matic = get_train_and_get_f1_acc(matic_data_dict_exp)

Training Accuracy: 0.5201665935992985
Validation Accuracy: 0.49842160645387584
Test Accuracy: 0.5014025245441796


Training F1 Score: 0.44046827871785693
Validation F1 Score: 0.4491525423728814
Test F1 Score: 0.3471074380165289


In [22]:

# # Compute timestamp-level representations for test set
# test_repr_tl = model.encode(test_data)  # n_instances x n_timestamps x output_dims

# # Compute instance-level representations for test set
# test_repr_il = model.encode(test_data, encoding_window='full_series')  # n_instances x output_dims

# # Sliding inference for test set
# test_repr_si = model.encode(
#     test_data,
#     casual=True,
#     sliding_length=1,
#     sliding_padding=50
# )  # n_instances x n_timestamps x output_dims
# # (The timestamp t's representation vector is computed using the observations located in [t-50, t])

In [23]:
# data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = datautils.load_forecast_csv("ETTh2")
# train_data = data[:, train_slice]
        
# # Train a TS2Vec model
# model = TS2Vec(
#     input_dims=1,
#     device=0,
#     output_dims=320
# )
# loss_log = model.fit(
#     train_data,
#     verbose=True
# )

# # Compute timestamp-level representations for test set
# test_repr = model.encode(test_data)  # n_instances x n_timestamps x output_dims

# # Compute instance-level representations for test set
# test_repr = model.encode(test_data, encoding_window='full_series')  # n_instances x output_dims

# # Sliding inference for test set
# test_repr = model.encode(
#     test_data,
#     casual=True,
#     sliding_length=1,
#     sliding_padding=50
# )  # n_instances x n_timestamps x output_dims
# # (The timestamp t's representation vector is computed using the observations located in [t-50, t])